<a href="https://colab.research.google.com/github/laurelkeys/machine-learning/blob/master/assignment-4/Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and setup Drive

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
!pip install PyDrive > /dev/null 2>&1

In [0]:
import os
import zipfile

import numpy as np

from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

## Download and unzip data

In [0]:
# Enable link sharing for data.zip in your Drive and copy the link:
# https://drive.google.com/open?id=YOUR_FILE_ID

YOUR_FILE_ID = '1fd1iyyeOpwAleUGv7QH23ocucxCzK4xt'

In [0]:
download = gdrive.CreateFile({'id': YOUR_FILE_ID})

save_folder = "data" # /content/data
os.makedirs(save_folder, exist_ok=True)
download.GetContentFile(os.path.join(save_folder, "data.zip"))

In [117]:
!ls -lh data

total 92M
drwxr-xr-x 3 root root 4.0K Dec  3 15:03 content
-rw-r--r-- 1 root root  92M Dec  3 15:53 data.zip


In [0]:
zip_ref = zipfile.ZipFile("/content/data/data.zip", 'r')
zip_ref.extractall("/content/data/")
zip_ref.close()

In [0]:
PATH_TO_DATASET = os.path.join("data", "content", "data") # data/data.zip -> data/content/data/*

In [44]:
!du -sh data/data.zip
!du -sh data/content

92M	data/data.zip
356M	data/content


In [76]:
print("================")
for trajectory_folder in os.listdir("data/content/data"):
    if trajectory_folder != "results":        
        print(trajectory_folder + '/')
        data_folders = sorted([d for d in os.listdir(f"data/content/data/{trajectory_folder}")])
        print("├──", data_folders[0] + '/')  # 0
        print("├── ...")
        print("├──", data_folders[-2] + '/') # N_OF_TRAJECTORIES
        print("└──", data_folders[-1] + '/') # 'images'
        
        images_folders = sorted([d for d in os.listdir(f"data/content/data/{trajectory_folder}/images")])
        # print("    ├──", images_folders[0] + '/')
        # print("    ├── ...")
        # print("    └──", images_folders[-1] + '/')
        
        last_images_folder = sorted([d for d in os.listdir(f"data/content/data/{trajectory_folder}/images/{images_folders[-1]}")])
        # print("        ├──", last_images_folder[0] + '/')
        # print("        ├── ...")
        # print("        └──", last_images_folder[-1] + '/')

        print(f"Trajectories: {int(images_folders[-1]) - int(images_folders[0]) + 1}")
        print(f"Steps per trajectory: {int(last_images_folder[-1][:-4]) - int(last_images_folder[0][:-4]) + 1}")
        print("================")

BreakoutNoFrameskip-v4_PPO2_1000steps/
├── 00/
├── ...
├── 44/
└── images/
Trajectories: 45
Steps per trajectory: 1000
PongNoFrameskip-v4_PPO2_1000steps/
├── 00/
├── ...
├── 44/
└── images/
Trajectories: 45
Steps per trajectory: 1000


## Inspect values

In [0]:
# NOTE to change

TRAJECTORY_FOLDER = "BreakoutNoFrameskip-v4_PPO2_1000steps"
STEP_FOLDER = "00"

In [118]:
x = np.load(os.path.join(PATH_TO_DATASET, TRAJECTORY_FOLDER, STEP_FOLDER, "trajectory.npz"), mmap_mode='r')
x.files

['actions', 'observations', 'rewards', 'episode_starts']

In [0]:
from gym.envs.atari.atari_env import ACTION_MEANING

def print_trajectory_info(x):
    # actions taken
    print("- actions:", x['actions'].shape)
    print("  actions taken:", ', '.join([ACTION_MEANING[action] for action in set(x['actions'].reshape(-1))]))
    # path to the observed images
    print("- observations:", x['observations'].shape)
    print("  path:", os.path.dirname(x['observations'][0]) + '/')
    # reward for each step
    print("- rewards:", x['rewards'].shape)
    print("  reward values:", ', '.join([str(r) for r in set(x['rewards'].reshape(-1))]))
    # `done` value returned by env.step(action)
    print("- episode_starts:", x['episode_starts'].shape)
    print("  episode starts:", [i for i, ep_start in enumerate(x['episode_starts']) if ep_start])

In [116]:
print_trajectory_info(x)

- actions: (1000,)
  actions taken: NOOP, FIRE, UP, RIGHT
- observations: (1000,)
  path: data/BreakoutNoFrameskip-v4_PPO2_1000steps/images/00/
- rewards: (1000,)
  reward values: 0.0, 1.0
- episode_starts: (1000,)
  episode starts: [0, 245]
